Testing the creation of bullet arms with PyVista  
Previous tests indicate that the boolean union of the bullet pyramid and the bullet cylinder is causing VTK error. 

In [1]:
import psutil
import numpy as np
import pandas as pd
from multiprocess import Pool, get_context
from itertools import product
import os
import pyvista as pv
import random
from math import pi, sqrt, pow, acos, degrees, radians, sin, cos
from data.rosette_temp import Rosette
from data import helper_temp as helper
from copy import deepcopy
import miniball
import pymeshfix as mf
from decimal import *

In [2]:
import math
def round_up_to_decimal(number, decimals=0):
    """Rounds a number up to the nearest specified decimal place.

    Args:
        number: The number to round up.
        decimals: The number of decimal places to round to (default is 0).

    Returns:
        The rounded-up number.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimals must be an integer")
    if decimals < 0:
        raise ValueError("decimals must be non-negative")
    
    factor = 10 ** decimals
    return math.ceil(number * factor) / factor

def round_down_to_decimal(number, decimals=0):
    """Rounds a number down to the nearest specified decimal place.

    Args:
        number: The number to round down.
        decimals: The number of decimal places to round to (default is 0).

    Returns:
        The rounded-down number.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimals must be an integer")
    if decimals < 0:
        raise ValueError("decimals must be non-negative")
    
    factor = 10 ** decimals
    return math.floor(number * factor) / factor

In [ ]:
output_path = '/Users/josephko/research/ice3d/output/out_log.txt'
for i in range(100):
    # define parameters
    c = random.uniform(10, 200)
    c = round_up_to_decimal(c, 1)
    a_c_ratio = random.uniform(0.05, 0.8)
    a = c * a_c_ratio
    a = round_up_to_decimal(a, 1)
    f_r0 = random.uniform(0.75, 1.0)
    r0 = f_r0 * a
    r0 = round_up_to_decimal(r0, 1)
    hp = r0 # pyramid height
    f_h0 = random.uniform(0.2, 0.5)
    h0 = hp/4 # penetration depth
    h0 = round_up_to_decimal(h0, 1)
    n_arms = random.randint(4, 10)
    
    # create components
    cyl = pv.Cylinder(center=(0.0, 0.0, c+hp), direction=(0.0, 0.0, -1.0), 
                    radius=a, height=2*c, resolution=6, capping=True)
    pyr = pv.Cone(center=(0.0, 0.0, hp/2), direction=(0.0, 0.0, -1.0), 
                height=hp, radius=a, capping=True, angle=None, resolution=6)
    cyl = cyl.triangulate()
    pyr = pyr.triangulate()
    cyl_pts = cyl.points
    cyl_pts[abs(cyl_pts)<1e-10]=0.0 # replace small values with zeros
    cyl.points = cyl_pts
    pyr_pts = pyr.points
    pyr_pts[abs(pyr_pts)<1e-10]=0.0 # replace small values with zeros
    pyr.points = pyr_pts

    # add parameters to text file
    file = open(output_path, 'a')
    file.write(f'{a}, {c}, {hp}\n')
    file.close()
    
    # perform boolean union 
    bullet = cyl.boolean_union(pyr).triangulate()

In [3]:
# test with particular set of parameters 
a, c, hp = 75.2, 132.0, 56.7

# create components
cyl = pv.Cylinder(center=(0.0, 0.0, c+hp), direction=(0.0, 0.0, -1.0), 
                radius=a, height=2*c, resolution=6, capping=True)
pyr = pv.Cone(center=(0.0, 0.0, hp/2), direction=(0.0, 0.0, -1.0), 
            height=hp, radius=a, capping=True, angle=None, resolution=6)
cyl = cyl.triangulate()
pyr = pyr.triangulate()
cyl_pts = cyl.points
# cyl_pts[abs(cyl_pts)<1e-10]=0.0 # replace small values with zeros
cyl.points = cyl_pts
pyr_pts = pyr.points
# pyr_pts[abs(pyr_pts)<1e-10]=0.0 # replace small values with zeros
pyr.points = pyr_pts

# plot cyl and pyr
p = pv.Plotter()
p.add_mesh(cyl, color='blue', show_edges=True)
p.add_mesh(pyr, color='red', show_edges=True)
p.show()

Widget(value='<iframe src="http://localhost:52700/index.html?ui=P_0x17bb93b00_0&reconnect=auto" class="pyvista…

In [5]:
# save pyr and cyl as obj files
cyl_path = '/Users/josephko/research/ice3d/output/cyl.obj'
pyr_path =  '/Users/josephko/research/ice3d/output/pyr.obj'
cyl.save(cyl_path)
pyr.save(pyr_path)

In [ ]:
# merge cyl and pyr
bullet = cyl.boolean_union(pyr).triangulate()

# Create generic cylinder and save to test with trimesh

In [4]:
import pyvista as pv
cyl_generic = pv.Cylinder(center=(0.0, 0.0, 0.0), direction=(0.0, 0.0, -1.0), 
                    radius=10, height=100, resolution=6, capping=True)
cyl_generic = cyl_generic.triangulate()
cyl_generic.plot(style='wireframe')

Widget(value='<iframe src="http://localhost:58312/index.html?ui=P_0x30d1ab0e0_2&reconnect=auto" class="pyvista…

In [5]:
# save to data folder in smooth-mesh-boolean folder
savepath = '/Users/josephko/research/smooth-mesh-boolean/data/cyl_generic.obj'
cyl_generic.save(savepath)